In [2]:
import pandas as pd
import bertopic
from bertopic import BERTopic
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from umap import UMAP
from bertopic.vectorizers import ClassTfidfTransformer

import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz

c:\Users\wojma\anaconda3\envs\UNI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
news = pd.read_csv('all_articles.csv')

With News API, I was able to collect the titles, descriptions, URLs etc. of news articles, which are all collected in the 'all_articles.csv' <br>
However, the full content of the article wasn't collected and that's what I will be doing below with Beautiful Soup

In [4]:
def scrape_article(url, intro):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        paragraphs = soup.find_all('p')

        for para in paragraphs:
            if fuzz.partial_ratio(para.text, intro) > 80:
                article = ' '.join([p.text for p in paragraphs[paragraphs.index(para):]])
                return article

        return "Article content not found."

    except requests.RequestException as e:
        return f"Error fetching the page: {e}"

index_list = []
url_list = []
article_list = []

for index, row in news.iterrows():
    full_article = scrape_article(row['url'], row['description'])
    index_list.append(index)
    url_list.append(row['url'])
    article_list.append(full_article)
    print(f"URL: {row['url']}\nArticle:\n{full_article}\n\n")

URL: https://www.emerce.nl/achtergrond/wat-zijn-de-kansen-van-een-marktplaats-voor-deelvervoer
Article:
Alle opties voor deelvervoer – fietsen, scooters, auto’s – in één app. Goed idee, zou je denken, maar waarom is die er dan (nog?) niet. Een blik op de economische en technische realiteit.
 De trend van bezit naar gebruik zien we overal om ons heen. De deeleconomie is beter voor het milieu en goed voor de portemonnee van de gebruiker. Voor de consument is de huidige situatie van deelvervoer echter niet ideaal: je hebt noodgedwongen meerdere apps op je telefoon voor de verschillende deelvoertuigen, zoals steps, fietsen, bakfietsen, scooters, auto’s en openbaar vervoer. De oplossing lijkt eenvoudig: één app voor een marktplaats waarin alle deelmobiliteit samenkomt. Of is het toch niet zo eenvoudig? In dit artikel bekijk ik de kansen en de beperkingen van een winnende marktplaats voor deelvervoer. Eindhoven is een mooi voorbeeld. Hier heb je drie aanbieders van deelfietsen (Go Sharing, T

In [5]:
scraped_df = pd.DataFrame({
    'Index': index_list,
    'URL': url_list,
    'Article': article_list
})
scraped_df

,Index,URL,Article
0,0,https://www.emerce.nl/achtergrond/wat-zijn-de-...,"Alle opties voor deelvervoer – fietsen, scoote..."
1,1,https://www.trouw.nl/duurzame-100/klimaatconfl...,Je bent veganist maar je partner houdt van spa...
2,2,https://www.bnr.nl/nieuws/mobiliteit/10527255/...,"Autodeelfirma's MyWheels, SHARE NOW, GreenMobi..."
3,3,https://www.nieuwsblad.be/cnt/dmf20231008_9512...,De Antwerpse politie heeft het afgelopen weeke...
4,4,https://www.gva.be/cnt/dmf20231006_94011344,Lokale burgerbeweging Klimaan breidt haar vloo...
5,5,https://www.nieuwsblad.be/cnt/dmf20231031_9284...,De Antwerpse politie hield bij een actie tegen...
6,6,https://www.nieuwsblad.be/cnt/dmf20231011_9321...,De Antwerpse politie betrapte dinsdag een wink...
7,7,https://www.nieuwsblad.be/cnt/dmf20231018_9342...,Duizenden lopers rennen zondag door de stad ti...
8,8,https://www.ad.nl/rotterdam/politie-slachtoffe...,Article content not found.
9,9,https://www.ad.nl/rotterdam/gamen-om-te-leren-...,Article content not found.


In [6]:
scraped_df.to_csv('scraped_articles.csv', index=False)

There were some articles that the scraper didn't scrape, but I still had the URLs, therefore I decided to find the content and manually copy paste it to a copy of the CSV. That updated copy is called "scraped_articles_full.csv"

In [7]:
full_aricles_df = pd.read_csv('scraped_articles_full.csv', encoding='latin')
full_aricles_df

,Index,URL,Article
0,0,https://www.emerce.nl/achtergrond/wat-zijn-de-...,"Alle opties voor deelvervoer â fietsen, scoo..."
1,1,https://www.trouw.nl/duurzame-100/klimaatconfl...,Je bent veganist maar je partner houdt van spa...
2,2,https://www.bnr.nl/nieuws/mobiliteit/10527255/...,"Autodeelfirma's MyWheels, SHARE NOW, GreenMobi..."
3,3,https://www.nieuwsblad.be/cnt/dmf20231008_9512...,De Antwerpse politie heeft het afgelopen weeke...
4,4,https://www.gva.be/cnt/dmf20231006_94011344,Lokale burgerbeweging Klimaan breidt haar vloo...
5,5,https://www.nieuwsblad.be/cnt/dmf20231031_9284...,De Antwerpse politie hield bij een actie tegen...
6,6,https://www.nieuwsblad.be/cnt/dmf20231011_9321...,De Antwerpse politie betrapte dinsdag een wink...
7,7,https://www.nieuwsblad.be/cnt/dmf20231018_9342...,Duizenden lopers rennen zondag door de stad ti...
8,8,https://www.ad.nl/rotterdam/politie-slachtoffe...,Politie: slachtoffer scooterongeluk was niet o...
9,9,https://www.ad.nl/rotterdam/gamen-om-te-leren-...,Gamen om te leren over gevaren van criminelen ...


To collect the topics, I will clean and process the text and then find topics using BERTopic

In [9]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('dutch'))
    tokens = [token for token in tokens if token not in stop_words and not token.isnumeric()]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    cleaned_text = ' '.join(tokens)

    return cleaned_text

def analyze_topics(texts):
    texts = texts.apply(preprocess_text)
    ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)
    topic_model = BERTopic(ctfidf_model=ctfidf_model )
    model = BERTopic(verbose=True, language='dutch', embedding_model='paraphrase-multilingual-minilm-l12-v2', min_topic_size=2, n_gram_range = (1, 2), ctfidf_model=ctfidf_model)
    topics, _ = model.fit_transform(texts)
    
    freq = model.get_topic_info()
    print("Number of topics: {}".format(len(freq)))
    display(freq.head(20))

    return model, topics


model, topics = analyze_topics(full_aricles_df['Article'])

full_aricles_df['topic'] = topics

#model.visualize_topics()
model.visualize_barchart()

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.27s/it]
2023-12-12 12:32:41,397 - BERTopic - Transformed documents to Embeddings
2023-12-12 12:32:51,169 - BERTopic - Reduced dimensionality
2023-12-12 12:32:51,206 - BERTopic - Clustered reduced embeddings


Number of topics: 4


,Topic,Count,Name,Representation,Representative_Docs
0,-1,3,-1_fiets_ov_ov fiets_haverman,"[fiets, ov, ov fiets, haverman, zegt, station,...",[duizenden lopers rennen zondag stad tijdens d...
1,0,19,0_we_jaar_alle_onze,"[we, jaar, alle, onze, mensen, weer, twee, ter...",[anderhalf jaar geleden zwaaide meyrem almaci ...
2,1,5,1_lijn_haltes_vanaf_januari,"[lijn, haltes, vanaf, januari, reizigers, buss...",[lijn zet begin volgend jaar grootste stap uit...
3,2,3,2_fiets_ov_ov fiets_haverman,"[fiets, ov, ov fiets, haverman, zegt, de, fiet...",[felyx-scooter inspireerde nieuwe initiatief o...


In [10]:
topics = model.get_topic_info()
topic_df = pd.DataFrame(topics)
topic_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3,-1_fiets_ov_ov fiets_haverman,"[fiets, ov, ov fiets, haverman, zegt, station,...",[duizenden lopers rennen zondag stad tijdens d...
1,0,19,0_we_jaar_alle_onze,"[we, jaar, alle, onze, mensen, weer, twee, ter...",[anderhalf jaar geleden zwaaide meyrem almaci ...
2,1,5,1_lijn_haltes_vanaf_januari,"[lijn, haltes, vanaf, januari, reizigers, buss...",[lijn zet begin volgend jaar grootste stap uit...
3,2,3,2_fiets_ov_ov fiets_haverman,"[fiets, ov, ov fiets, haverman, zegt, de, fiet...",[felyx-scooter inspireerde nieuwe initiatief o...


In [11]:
topic_df.to_csv('topics_information.csv', index=False)

Now, I will run sentiment analysis on the topics found to determine what the attitude towards those topics is.

In [12]:
def analyze_sentiment(text):
    url = "http://text-processing.com/api/sentiment/"
    data = {'text': text, 'language': 'dutch'}

    response = requests.post(url, data=data)
    result = response.json()

    sentiment = result['label']
    confidence = result['probability'][sentiment]

    return sentiment, confidence

In [13]:
topic_df['sentiment'], topic_df['confidence'] = zip(*topic_df['Representative_Docs'].apply(analyze_sentiment))

print(topic_df[['Representative_Docs', 'sentiment', 'confidence']])

                                 Representative_Docs sentiment  confidence
0  [duizenden lopers rennen zondag stad tijdens d...       pos    0.546533
1  [anderhalf jaar geleden zwaaide meyrem almaci ...       pos    0.519667
2  [lijn zet begin volgend jaar grootste stap uit...       pos    0.595220
3  [felyx-scooter inspireerde nieuwe initiatief o...       pos    0.593750


Now, I will determine what those topics mean and what they exactly are by translating and analysing the 'Representation' found by BERTopic, therefore the words representing these specific topics.